In [1]:
import os

# transformers가 TensorFlow/Flax 쪽을 아예 무시하게 강제 설정
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"
os.environ["USE_TF"] = "0"   # 추가 안전장치
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # TF 관련 경고 숨기기

In [2]:
import sys
import subprocess

def pip_install(pkg):
    print(f"📦 Installing {pkg} ...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# -------------------------
# 1) 버전 확인
# -------------------------

print("=== Python Version ===")
print(sys.version)

# -------------------------
# 2) 패키지 설치 확인 + 자동 설치
# -------------------------

required = [
    "torch",
    "transformers",
    "accelerate",
    "datasets",
    "scikit-learn"
]

for pkg in required:
    try:
        __import__(pkg.replace("-", "_"))
        print(f"✔ {pkg} already installed")
    except ModuleNotFoundError:
        pip_install(pkg)

print("=== 패키지 체크 완료 ===")

=== Python Version ===
3.11.14 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 18:30:03) [MSC v.1929 64 bit (AMD64)]
✔ torch already installed
✔ transformers already installed
✔ accelerate already installed
✔ datasets already installed
📦 Installing scikit-learn ...
=== 패키지 체크 완료 ===


In [3]:
import torch, transformers, accelerate

print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("CUDA available:", torch.cuda.is_available())

Torch: 2.3.1
Transformers: 4.39.3
Accelerate: 0.30.1
CUDA available: False


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "madatnlp/km-bert"

print("Loading KM-BERT tokenizer/model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

text = "머리가 깨질 것처럼 아파요"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

print("input_ids shape:", inputs["input_ids"].shape)
print("last_hidden_state shape:", outputs.last_hidden_state.shape)
print("🎉 KM-BERT forward pass 성공")

Loading KM-BERT tokenizer/model...


C:\Users\jang8\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


input_ids shape: torch.Size([1, 10])
last_hidden_state shape: torch.Size([1, 10, 768])
🎉 KM-BERT forward pass 성공


In [5]:
import pandas as pd

csv_path = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\medical_colloquial_sample.csv"
df = pd.read_csv(csv_path)

print(df.head())
print("컬럼:", df.columns.tolist())
print("데이터 개수:", len(df))

          colloquial normalized_symptom department
0    머리가 깨질 것처럼 아파요.                 두통        신경과
1   한쪽 머리가 지끈지끈 쑤셔요.                편두통        신경과
2  머리 전체가 조이는 느낌이에요.             긴장성 두통        신경과
3   머릿속이 울리는 듯이 아파요.                 두통        신경과
4  아침마다 머리가 무겁고 띵해요.                 두통        신경과
컬럼: ['colloquial', 'normalized_symptom', 'department']
데이터 개수: 435


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset

# 현재 df 컬럼에 맞게 증상/텍스트 컬럼 이름 자동 설정
symptom_col = "normalized_symptom" if "normalized_symptom" in df.columns else "label"
text_col    = "colloquial"         if "colloquial" in df.columns         else "text"

print("현재 df 컬럼:", list(df.columns))
print("증상 컬럼(symptom_col):", symptom_col)
print("텍스트 컬럼(text_col):", text_col)

# 1) 우리가 UT에서 다루고 싶은 진료과 후보
target_depts = [
    "이비인후과",
    "정형외과",
    "피부과",
    "신경과",
    "내과",
    "소화기내과",
]

# 실제 데이터에 존재하는 진료과만 사용
dept_counts = df["department"].value_counts()
valid_target_depts = [d for d in target_depts if d in dept_counts.index]

print("\n사용 가능한 타깃 진료과:", valid_target_depts)
print(dept_counts[valid_target_depts])

# 2) 각 진료과마다 자주 나오는 증상 상위 K개씩 뽑기
MIN_SYMPTOM_COUNT = 3   # 최소 3개 이상 있는 증상만
TOP_K_PER_DEPT    = 5   # 진료과당 최대 5개 증상

ut_labels = set()

for dept in valid_target_depts:
    sub = df[df["department"] == dept]
    vc = sub[symptom_col].value_counts()

    chosen = vc[vc >= MIN_SYMPTOM_COUNT].head(TOP_K_PER_DEPT).index
    print(f"\n[{dept}] 선택된 증상들:", list(chosen))
    ut_labels.update(chosen)

ut_labels = list(ut_labels)
print("\nUT용 최종 증상 개수:", len(ut_labels))
print("UT용 증상 목록:", ut_labels)

# 3) 이 증상들만 모은 UT용 데이터프레임
df_ut = df[df[symptom_col].isin(ut_labels)].copy()
print("\nUT용 데이터 샘플 수:", len(df_ut))

# 4) 컬럼 이름을 학습용으로 통일: text / label / department
df_ut = df_ut.rename(
    columns={
        text_col: "text",
        symptom_col: "label"
    }
)

# 5) 라벨 인코딩
le = LabelEncoder()
df_ut["label_id"] = le.fit_transform(df_ut["label"])

print("라벨 개수:", df_ut["label_id"].nunique())
print("라벨 분포:")
print(df_ut["label"].value_counts())

# 6) train / eval 분리
train_df, eval_df = train_test_split(
    df_ut,
    test_size=0.2,
    stratify=df_ut["label_id"],
    random_state=42,
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
eval_ds  = Dataset.from_pandas(eval_df.reset_index(drop=True))

print("\ntrain / eval 크기:", len(train_ds), len(eval_ds))

현재 df 컬럼: ['colloquial', 'normalized_symptom', 'department']
증상 컬럼(symptom_col): normalized_symptom
텍스트 컬럼(text_col): colloquial

사용 가능한 타깃 진료과: ['이비인후과', '정형외과', '피부과', '신경과', '내과', '소화기내과']
department
이비인후과    28
정형외과     50
피부과      41
신경과      94
내과       32
소화기내과    42
Name: count, dtype: int64

[이비인후과] 선택된 증상들: ['이명']

[정형외과] 선택된 증상들: []

[피부과] 선택된 증상들: ['두피 통증']

[신경과] 선택된 증상들: ['두통', '어지럼증', '긴장성 두통', '편두통', '뇌 안개']

[내과] 선택된 증상들: ['전신 피로']

[소화기내과] 선택된 증상들: ['복통', '소화불량', '위염']

UT용 최종 증상 개수: 11
UT용 증상 목록: ['두통', '어지럼증', '뇌 안개', '두피 통증', '복통', '긴장성 두통', '편두통', '전신 피로', '위염', '이명', '소화불량']

UT용 데이터 샘플 수: 63
라벨 개수: 11
라벨 분포:
label
두통        24
어지럼증       5
복통         5
소화불량       5
편두통        4
긴장성 두통     4
뇌 안개       4
두피 통증      3
이명         3
위염         3
전신 피로      3
Name: count, dtype: int64

train / eval 크기: 50 13


In [7]:
from sklearn.preprocessing import LabelEncoder

print("=== 원래 df_ut 라벨 분포 ===")
print(df_ut["label"].value_counts())

# 1) 각 라벨이 몇 개씩 있는지 확인
label_counts = df_ut["label"].value_counts()
max_count = label_counts.max()   # 가장 많은 라벨 개수만큼 맞춰줄 거야

balanced_list = []

for label, group in df_ut.groupby("label"):
    # 각 라벨을 max_count 개수만큼 복원추출(중복 허용)로 샘플링
    balanced = group.sample(n=max_count, replace=True, random_state=42)
    balanced_list.append(balanced)

df_ut_balanced = (
    pd.concat(balanced_list)
      .sample(frac=1, random_state=42)  # 섞기
      .reset_index(drop=True)
)

df_ut = df_ut_balanced.copy()

print("\n=== 오버샘플링 후 df_ut 라벨 분포 ===")
print(df_ut["label"].value_counts())

# 라벨 인코딩 다시
le = LabelEncoder()
df_ut["label_id"] = le.fit_transform(df_ut["label"])

=== 원래 df_ut 라벨 분포 ===
label
두통        24
어지럼증       5
복통         5
소화불량       5
편두통        4
긴장성 두통     4
뇌 안개       4
두피 통증      3
이명         3
위염         3
전신 피로      3
Name: count, dtype: int64

=== 오버샘플링 후 df_ut 라벨 분포 ===
label
두통        24
이명        24
두피 통증     24
편두통       24
복통        24
위염        24
긴장성 두통    24
전신 피로     24
뇌 안개      24
소화불량      24
어지럼증      24
Name: count, dtype: int64


In [8]:
# 6번 셀: 토크나이징
def tokenize_batch(batch):
    encoded = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=64,
    )
    encoded["labels"] = batch["label_id"]
    return encoded

train_ds_tok = train_ds.map(tokenize_batch, batched=True)
eval_ds_tok  = eval_ds.map(tokenize_batch, batched=True)

# Trainer에 필요 없는 컬럼 제거
cols_keep = ["input_ids", "attention_mask", "token_type_ids", "labels"]
cols_drop = [c for c in train_ds_tok.column_names if c not in cols_keep]

train_ds_tok = train_ds_tok.remove_columns(cols_drop)
eval_ds_tok  = eval_ds_tok.remove_columns(cols_drop)

train_ds_tok[0]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

{'input_ids': [2,
  605,
  9,
  684,
  21,
  353,
  16,
  169,
  221,
  239,
  5,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
# num_labels = df_filtered["label_id"].nunique()   # (X)
num_labels = df_ut["label_id"].nunique()           # (O)

model_cls = AutoModelForSequenceClassification.from_pretrained(
    "madatnlp/km-bert",
    num_labels=num_labels,
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "macro_f1": f1_score(labels, preds, average="macro")
    }

training_args = TrainingArguments(
    output_dir="./kmbert_mednorm",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    report_to="none",
)

trainer = Trainer(
    model=model_cls,
    args=training_args,
    train_dataset=train_ds_tok,
    eval_dataset=eval_ds_tok,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\jang8\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at madatnlp/km-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\jang8\anaconda3\Lib\site-packages\accelerate\accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, s

Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,No log,2.025290,0.384615,0.061728
2,No log,1.959257,0.384615,0.061728
3,No log,1.902410,0.384615,0.061728
4,No log,1.849226,0.384615,0.061728
5,No log,1.815848,0.384615,0.061728


TrainOutput(global_step=35, training_loss=1.972869655064174, metrics={'train_runtime': 60.6925, 'train_samples_per_second': 4.119, 'train_steps_per_second': 0.577, 'total_flos': 8222884896000.0, 'train_loss': 1.972869655064174, 'epoch': 5.0})

In [10]:
import torch

def predict_symptom(texts):
    if isinstance(texts, str):
        texts = [texts]

    inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64,
    )

    with torch.no_grad():
        logits = model_cls(**inputs).logits

    pred_ids = logits.argmax(dim=-1).tolist()
    pred_labels = le.inverse_transform(pred_ids)

    for t, lab in zip(texts, pred_labels):
        print(f"[{t}] -> 예측 정규화 증상: {lab}")

# 테스트 몇 개 해보기
test_texts = [
    "머리가 깨질 것처럼 아파요",
    "한쪽 머리가 지끈지끈 쑤셔요",
    "속이 메스껍고 토할 것 같아요",
    "허리가 뻐근하고 욱신거려요",
]

predict_symptom(test_texts)

[머리가 깨질 것처럼 아파요] -> 예측 정규화 증상: 두통
[한쪽 머리가 지끈지끈 쑤셔요] -> 예측 정규화 증상: 두통
[속이 메스껍고 토할 것 같아요] -> 예측 정규화 증상: 두통
[허리가 뻐근하고 욱신거려요] -> 예측 정규화 증상: 두통


✔ 1) KM-BERT 모델 로드
✔ 2) 토크나이저 준비
✔ 3) label encoder 준비
✔ 4) predict_symptom(texts) 함수까지 완성

In [11]:
# df_ut 안에 text / label / label_id / department 가 있다고 가정

# 1) 증상(label)별로 가장 많이 등장한 진료과를 대표값으로 사용
symptom_to_dept = (
    df_ut.groupby("label")["department"]
         .agg(lambda x: x.value_counts().idxmax())
         .to_dict()
)

print("증상→진료과 매핑 예시 5개:")
for k in list(symptom_to_dept.keys())[:5]:
    print(k, "→", symptom_to_dept[k])

증상→진료과 매핑 예시 5개:
긴장성 두통 → 신경과
뇌 안개 → 신경과
두통 → 신경과
두피 통증 → 피부과
복통 → 소화기내과


In [12]:
import torch

def predict_symptom_single(text: str) -> str:
    """한 문장에 대해 정규화된 증상 라벨만 반환"""
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64,
    )

    with torch.no_grad():
        logits = model_cls(**inputs).logits

    pred_id = logits.argmax(dim=-1).item()
    pred_label = le.inverse_transform([pred_id])[0]
    return pred_label


def mediq_reply(user_text: str, body_part: str | None = None) -> dict:
    """
    구어체 증상을 입력하면:
    - 정규화된 증상
    - CSV 기반 진료과
    - 간단한 설명(템플릿)
    - 후속 질문
    을 반환하는 함수
    """

    symptom = predict_symptom_single(user_text)

    # CSV에서 뽑은 매핑만 사용
    dept = symptom_to_dept.get(symptom, "진료과 정보 없음")

    # 설명과 질문은 '의미를 꾸며주는 문장 템플릿' 수준으로만
    desc = f"입력하신 표현은 '{symptom}' 범주로 정규화되었습니다. 이 증상은 보통 '{dept}'에서 주로 다루는 증상입니다."
    if body_part:
        desc += f" 선택하신 부위는 '{body_part}'입니다."

    follow_up = f"이 '{symptom}' 증상은 언제부터, 얼마나 자주 나타나는지 알려주실 수 있을까요?"

    return {
        "입력문장": user_text,
        "정규화_증상": symptom,
        "추천_진료과": dept,
        "설명": desc,
        "추가질문": follow_up,
    }

# 테스트
print(mediq_reply("머리가 깨질 것처럼 아파요"))

{'입력문장': '머리가 깨질 것처럼 아파요', '정규화_증상': '두통', '추천_진료과': '신경과', '설명': "입력하신 표현은 '두통' 범주로 정규화되었습니다. 이 증상은 보통 '신경과'에서 주로 다루는 증상입니다.", '추가질문': "이 '두통' 증상은 언제부터, 얼마나 자주 나타나는지 알려주실 수 있을까요?"}


In [13]:
predict_symptom(["배가 아파요", "토할 것 같아요", "속이 쓰려요"])

[배가 아파요] -> 예측 정규화 증상: 두통
[토할 것 같아요] -> 예측 정규화 증상: 두통
[속이 쓰려요] -> 예측 정규화 증상: 두통


In [14]:
print("=== df_ut 샘플 5개 ===")
print(df_ut.head())

print("\n=== df_ut 라벨 분포 ===")
print(df_ut["label"].value_counts())

=== df_ut 샘플 5개 ===
                      text  label department  label_id
0        고개를 숙이면 머리가 찌릿해요.     두통        신경과         2
1     귀에서 물 소리 같은 게 계속 나요.     이명      이비인후과         8
2        머리를 감을 때 두피가 아파요.  두피 통증        피부과         3
3         한쪽 머리가 지끈지끈 쑤셔요.    편두통        신경과        10
4  배에서 찌릿하게 전기가 흐르는 듯 아파요.     복통      소화기내과         4

=== df_ut 라벨 분포 ===
label
두통        24
이명        24
두피 통증     24
편두통       24
복통        24
위염        24
긴장성 두통    24
전신 피로     24
뇌 안개      24
소화불량      24
어지럼증      24
Name: count, dtype: int64


In [15]:
import os
import json
import pandas as pd

# 📌 최상위 폴더 (여기까지가 '120.초거대AI 사전학습용 헬스케어 질의응답 데이터')
base_dir = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터"

# 📌 너가 요청한 경로 구조 반영한 버전
json_dirs = [
    r"3.개방데이터\1.데이터\Training\01.원천데이터\TS\1.질문",
    r"3.개방데이터\1.데이터\Training\01.원천데이터\TS\2.답변",

    r"3.개방데이터\1.데이터\Training\02.라벨링데이터\TL\1.질문",
    r"3.개방데이터\1.데이터\Training\02.라벨링데이터\TL\2.답변",

    r"3.개방데이터\1.데이터\Validation\01.원천데이터\VS\1.질문",
    r"3.개방데이터\1.데이터\Validation\01.원천데이터\VS\2.답변",

    r"3.개방데이터\1.데이터\Validation\02.라벨링데이터\VL\1.질문",
    r"3.개방데이터\1.데이터\Validation\02.라벨링데이터\VL\2.답변",
]

json_paths = []

print("=== JSON 폴더 스캔 시작 ===\n")

for rel_path in json_dirs:
    target = os.path.join(base_dir, rel_path)

    if not os.path.exists(target):
        print(f"⚠ 경로 없음 → {target}")
        continue

    print(f"📂 스캔 중: {target}")

    for root, dirs, files in os.walk(target):
        for fname in files:
            if fname.lower().endswith(".json"):
                json_paths.append(os.path.join(root, fname))

print("\n=== 📄 JSON 수집 완료 ===")
print("총 JSON 파일 개수:", len(json_paths))
print("\n📌 예시 10개:")
for p in json_paths[:10]:
    print(" -", p)

=== JSON 폴더 스캔 시작 ===

📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\1.질문
📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Training\01.원천데이터\TS\2.답변
📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Training\02.라벨링데이터\TL\1.질문
📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Training\02.라벨링데이터\TL\2.답변
📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Validation\01.원천데이터\VS\1.질문
📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Validation\01.원천데이터\VS\2.답변
📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Validation\02.라벨링데이터\VL\1.질문
📂 스캔 중: C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI 사전학습용 헬스케어 질의응답 데이터\3.개방데이터\1.데이터\Validation\02.라벨링데이터\VL\2.답변

=== 📄 JSON 수집 완료 ===
총 JSON 파일 개수: 6993458



In [16]:
test_paths = json_paths[:1000]

rows = []

for path in test_paths:
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        question = data.get("question", "")
        answer = data.get("answer", "")
        disease_category = data.get("disease_category", "")
        disease_name = data.get("disease_name", {}).get("kor", "")
        
        rows.append({
            "colloquial": question,
            "answer": answer,
            "category": disease_category,
            "disease": disease_name,
            "source": path
        })
    except:
        continue

df_test = pd.DataFrame(rows)
df_test.head()

,colloquial,answer,category,disease,source
0,"HIV 감염을 의심하고 있는데, 어떤 곳에서 검진을 받아야 할까요?",,감염성질환,HIV 감염,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
1,성 접촉 이후 HIV 감염으로 인한 증상이 어떻게 나타나는지 알고 싶어요.,,감염성질환,HIV 감염,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
2,HIV 감염을 진단하기 위해 사용되는 주요 검사 방법은 무엇인가요?,,감염성질환,HIV 감염,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
3,HIV 감염을 의심할 때 어떤 검사를 통해 진단을 받아야 할까요?,,감염성질환,HIV 감염,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
4,HIV 감염 검사는 백혈구 수치 감소로 인해 권장되는 검사인가요?,,감염성질환,HIV 감염,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...


In [17]:
import random
import json
import pandas as pd

# 전체 json_paths 중에서 1000개 랜덤 선택
test_paths = random.sample(json_paths, min(1000, len(json_paths)))

rows = []

for path in test_paths:
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        question = data.get("question", "")
        answer = data.get("answer", "")
        disease_category = data.get("disease_category", "")

        # disease_name dict(eng/kor) 또는 문자열 케이스 자동처리
        raw_dis = data.get("disease_name", "")
        if isinstance(raw_dis, dict):
            disease_name = raw_dis.get("kor", "")
        else:
            disease_name = raw_dis

        rows.append({
            "colloquial": question.strip(),
            "answer": answer.strip(),
            "category": disease_category.strip(),
            "disease": disease_name.strip(),
            "source": path
        })
    except:
        continue

df_test = pd.DataFrame(rows)
df_test.head()

,colloquial,answer,category,disease,source
0,림프종 치료에는 어떤 종류의 운동 치료가 효과적인가요?,,종양혈액질환,림프종,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
1,손목 수근관 증후군의 증상을 자세히 알고 싶어요.,,근골격질환,손목 수근관 증후군,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
2,폐결핵 예방을 위해 어떤 식이 관련 습관을 지켜야 할까요?,,감염성질환,결핵,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
3,두통과 같은 질환을 예방하는 방법이 있을까요?,,응급질환,두통,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...
4,산후 회복을 위한 식이와 생활 습관에 대해 알려주세요.,,여성질환,유산,C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...


In [18]:
print(df_test.shape)
print(df_test["disease"].value_counts().head(20))

(362, 5)
disease
고혈압                 7
패혈증                 4
거북목 증후군             4
안검하수                3
발육부전성 좌심 증후군        3
소장암                 3
구순열                 3
뇌하수체 양성 및 악성 신생물    3
헤르페스 바이러스 감염        3
손목 수근관 증후군          3
자궁경부암               3
뇌경색                 3
척추측만증               3
면역혈소판감소증            3
치매                  3
천식                  3
골다공증                3
피부암                 3
간염                  3
두통                  3
Name: count, dtype: int64


In [35]:
import json
import pandas as pd

CHUNK_SIZE = 100000   # 10만개씩 CSV 저장
rows = []
file_index = 1
processed = 0

print(f"총 JSON 개수: {len(json_paths)}")
print("=== 전체 파싱 시작 ===\n")

for idx, path in enumerate(json_paths):

    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)

        question = data.get("question", "")
        answer = data.get("answer", "")
        disease_category = data.get("disease_category", "")

        # disease_name이 dict일 수도 있기 때문에 처리
        raw_dis = data.get("disease_name", "")
        if isinstance(raw_dis, dict):
            disease_name = raw_dis.get("kor", "")
        else:
            disease_name = raw_dis

        rows.append({
            "colloquial": question.strip(),
            "answer": answer.strip(),
            "category": disease_category.strip(),
            "disease": disease_name.strip(),
            "source": path
        })

    except:
        # 깨진 json 파일은 skip
        continue

    processed += 1

    # 10만 개 모였을 때 CSV로 저장
    if processed % CHUNK_SIZE == 0:
        df_chunk = pd.DataFrame(rows)
        save_path = f"all_data_part_{file_index}.csv"
        df_chunk.to_csv(save_path, index=False, encoding="utf-8-sig")

        print(f"💾 저장 완료: {save_path} ({processed}/{len(json_paths)} 완료)")
        
        rows = []  # 리스트 초기화
        file_index += 1

# 마지막에 남은 데이터 저장
if len(rows) > 0:
    df_chunk = pd.DataFrame(rows)
    save_path = f"all_data_part_{file_index}.csv"
    df_chunk.to_csv(save_path, index=False, encoding="utf-8-sig")
    print(f"💾 마지막 저장 완료: {save_path}")

print("\n=== 전체 파싱 완료! ===")

총 JSON 개수: 6993458
=== 전체 파싱 시작 ===

💾 저장 완료: all_data_part_1.csv (100000/6993458 완료)
💾 저장 완료: all_data_part_2.csv (200000/6993458 완료)
💾 저장 완료: all_data_part_3.csv (300000/6993458 완료)
💾 저장 완료: all_data_part_4.csv (400000/6993458 완료)
💾 저장 완료: all_data_part_5.csv (500000/6993458 완료)
💾 저장 완료: all_data_part_6.csv (600000/6993458 완료)
💾 저장 완료: all_data_part_7.csv (700000/6993458 완료)
💾 저장 완료: all_data_part_8.csv (800000/6993458 완료)
💾 저장 완료: all_data_part_9.csv (900000/6993458 완료)
💾 저장 완료: all_data_part_10.csv (1000000/6993458 완료)
💾 저장 완료: all_data_part_11.csv (1100000/6993458 완료)
💾 저장 완료: all_data_part_12.csv (1200000/6993458 완료)
💾 저장 완료: all_data_part_13.csv (1300000/6993458 완료)
💾 저장 완료: all_data_part_14.csv (1400000/6993458 완료)
💾 저장 완료: all_data_part_15.csv (1500000/6993458 완료)
💾 저장 완료: all_data_part_16.csv (1600000/6993458 완료)
💾 저장 완료: all_data_part_17.csv (1700000/6993458 완료)
💾 저장 완료: all_data_part_18.csv (1800000/6993458 완료)
💾 저장 완료: all_data_part_19.csv (1900000/6993458 완료)
💾 저장 완료: all

In [ ]:
print(len(json_paths))  # 6993458 나오는지 확인

In [19]:
import os
import pandas as pd

# CSV들이 있는 폴더
data_dir = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터"

# 모든 CSV 파일 불러오기
csv_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]

dfs = []
for f in csv_files:
    path = os.path.join(data_dir, f)
    print("병합 중 →", f)
    df = pd.read_csv(path)
    dfs.append(df)

# 최종 하나로 합치기
final_df = pd.concat(dfs, ignore_index=True)

# 저장
final_path = os.path.join(data_dir, "merged_all_data.csv")
final_df.to_csv(final_path, index=False, encoding="utf-8-sig")

print("\n🎉 병합 완료!")
print("총 데이터 수:", len(final_df))

병합 중 → all_data_part_1.csv
병합 중 → all_data_part_10.csv
병합 중 → all_data_part_11.csv
병합 중 → all_data_part_12.csv
병합 중 → all_data_part_13.csv
병합 중 → all_data_part_14.csv
병합 중 → all_data_part_15.csv
병합 중 → all_data_part_16.csv
병합 중 → all_data_part_17.csv
병합 중 → all_data_part_18.csv
병합 중 → all_data_part_19.csv
병합 중 → all_data_part_2.csv
병합 중 → all_data_part_20.csv
병합 중 → all_data_part_21.csv
병합 중 → all_data_part_22.csv
병합 중 → all_data_part_23.csv
병합 중 → all_data_part_24.csv
병합 중 → all_data_part_25.csv
병합 중 → all_data_part_26.csv
병합 중 → all_data_part_3.csv
병합 중 → all_data_part_4.csv
병합 중 → all_data_part_5.csv
병합 중 → all_data_part_6.csv
병합 중 → all_data_part_7.csv
병합 중 → all_data_part_8.csv
병합 중 → all_data_part_9.csv
병합 중 → health_colloquial_raw.csv
병합 중 → medical_colloquial_sample.csv

🎉 병합 완료!
총 데이터 수: 2625131


In [1]:
import pandas as pd

df = pd.read_csv(r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\merged_all_data.csv")

print(df.head())
print(df.columns.tolist())
print("데이터 수:", len(df))

C:\Users\jang8\AppData\Local\Temp\ipykernel_15056\3302758387.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\merged_all_data.csv")


                                  colloquial  answer category disease  \
0      HIV 감염을 의심하고 있는데, 어떤 곳에서 검진을 받아야 할까요?     NaN    감염성질환  HIV 감염   
1  성 접촉 이후 HIV 감염으로 인한 증상이 어떻게 나타나는지 알고 싶어요.     NaN    감염성질환  HIV 감염   
2      HIV 감염을 진단하기 위해 사용되는 주요 검사 방법은 무엇인가요?     NaN    감염성질환  HIV 감염   
3       HIV 감염을 의심할 때 어떤 검사를 통해 진단을 받아야 할까요?     NaN    감염성질환  HIV 감염   
4       HIV 감염 검사는 백혈구 수치 감소로 인해 권장되는 검사인가요?     NaN    감염성질환  HIV 감염   

                                              source source_file subject  \
0  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
1  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
2  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
3  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   
4  C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\120.초거대AI...         NaN     NaN   

  sentence normalized_symptom department  
0      NaN                NaN        NaN  
1      NaN        

In [2]:
csv_path = r"C:\Users\jang8\OneDrive\바탕 화면\졸작\데이터\merged_all_data.csv"
df = pd.read_csv(csv_path)

# 1) 텍스트를 하나의 컬럼으로 통합
#    - Q&A 계열: 'colloquial' 채워져 있음
#    - 구어체 정규화 계열: 'sentence' 채워져 있음
df["text"] = df["colloquial"].fillna(df["sentence"])

# 2) normalized_symptom 이 있는 행만 사용 (정규화 학습용)
df_sym = df[~df["normalized_symptom"].isna()].copy()

# 3) 필요한 컬럼만 남기기
df_sym = df_sym[["text", "normalized_symptom", "department"]]

print("정규화 학습 데이터 수:", len(df_sym))
print(df_sym.head())
print(df_sym["normalized_symptom"].value_counts().head(20))

C:\Users\jang8\AppData\Local\Temp\ipykernel_15056\2316680732.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


정규화 학습 데이터 수: 435
                      text normalized_symptom department
2624696    머리가 깨질 것처럼 아파요.                 두통        신경과
2624697   한쪽 머리가 지끈지끈 쑤셔요.                편두통        신경과
2624698  머리 전체가 조이는 느낌이에요.             긴장성 두통        신경과
2624699   머릿속이 울리는 듯이 아파요.                 두통        신경과
2624700  아침마다 머리가 무겁고 띵해요.                 두통        신경과
normalized_symptom
두통                    24
normalized_symptom     8
하복부통증                  6
어지럼증                   5
소화불량                   5
흉통                     5
복통                     5
뇌 안개                   4
편두통                    4
긴장성 두통                 4
호흡곤란                   4
말초신경병증                 4
만성 피로                  3
이명                     3
수면장애                   3
흉부 압박감                 3
가래                     3
심계항진                   3
좌골신경통                  3
질염                     3
Name: count, dtype: int64


In [3]:
import torch

print("CUDA 가능?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU 개수:", torch.cuda.device_count())
    print("0번 GPU 이름:", torch.cuda.get_device_name(0))

CUDA 가능? False


In [4]:
import torch
print("torch version:", torch.__version__)
print("torch.cuda version in torch:", torch.version.cuda)
print("GPU 개수:", torch.cuda.device_count())

torch version: 2.3.1
torch.cuda version in torch: None
GPU 개수: 0
